In [3]:
from cnn_models_utils import load_model, get_probabilities, evaluate, train_with_earlystop
from CIFAR_CNN import CIFAR_CNN_Classifier
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import Compose, ToTensor, Normalize, RandomRotation

from torch import save, no_grad, argmax, load


from MNIST_CNN_earlystopping import Net, Mnist_CNN_Classifier

In [8]:
transform = Compose([ToTensor(), Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_dataset = CIFAR10(root='../data', train=False, download=True, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = load_model("models/CNN_cifar_base_softmax.torch",CIFAR_CNN_Classifier())
# probs = get_probabilities(test_dataloader,model)
# print(probs[0:1])


# Rotate images by 90 degrees 
transform = Compose([ToTensor(), RandomRotation (degrees = 90) ,Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test__rotated_dataset = CIFAR10(root='../data', train=False, download=True, transform=transform)
test_Rrotated_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

evaluate(test_Rrotated_dataloader,model,classes = classes)

Files already downloaded and verified
Files already downloaded and verified
Accuracy for class: plane is 64.2 %
Accuracy for class: car is 77.7 %
Accuracy for class: bird is 39.1 %
Accuracy for class: cat is 52.6 %
Accuracy for class: deer is 36.9 %
Accuracy for class: dog is 49.2 %
Accuracy for class: frog is 52.1 %
Accuracy for class: horse is 57.0 %
Accuracy for class: ship is 67.0 %
Accuracy for class: truck is 55.6 %
Total Accuracy: 55.1 %


In [7]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images)
        predictions = argmax(outputs, 1) 
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname} is {accuracy:.1f} %')

total_accuracy = sum(correct_pred.values()) / sum(total_pred.values())
print(f'Total Accuracy: {total_accuracy * 100:.1f} %')

Accuracy for class: plane is 63.6 %
Accuracy for class: car is 76.8 %
Accuracy for class: bird is 41.1 %
Accuracy for class: cat is 51.8 %
Accuracy for class: deer is 33.9 %
Accuracy for class: dog is 48.7 %
Accuracy for class: frog is 50.4 %
Accuracy for class: horse is 58.2 %
Accuracy for class: ship is 66.0 %
Accuracy for class: truck is 54.3 %
Total Accuracy: 54.5 %


# test mnist non-local load and train

In [39]:
from torchvision.datasets.mnist import MNIST 
train_dataset = MNIST(root='./data', train=True, download=True, transform=ToTensor())
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# split train to train and val
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

#val
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

test_dataset = MNIST(root='./data', train=False, download=False, transform=ToTensor())
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

m =  Mnist_CNN_Classifier()
m =  Net()

model = train_with_earlystop(dataloader = train_dataloader, model = m, lr= 0.02,
                                    , patience=3, dataloader_val = val_dataloader , num_epochs= 20)

evaluate(test_dataloader, model)

Training progress:  85%|████████████▊  | 17/20 [04:14<00:44, 14.95s/it, Epoch Loss=1.52, Validation Loss=1.51, Best val loss=1.51, Patience Counter=2]


Early stopping triggered!
Accuracy for class: 0 is 97.3 %
Accuracy for class: 1 is 97.2 %
Accuracy for class: 2 is 96.5 %
Accuracy for class: 3 is 95.5 %
Accuracy for class: 4 is 95.1 %
Accuracy for class: 5 is 94.1 %
Accuracy for class: 6 is 95.4 %
Accuracy for class: 7 is 95.1 %
Accuracy for class: 8 is 91.7 %
Accuracy for class: 9 is 93.9 %
Total Accuracy: 95.2 %
